# SNOMED

SNOMED CT is a standarised clinical terminology consisting of >350,000 unique concepts. It is owned, maintained and distributed by SNOMED International.

Please visit and explore https://www.snomed.org/ to find out further information about the various SNOMED CT products and services which they offer.

--------

All raw files from SNOMED can be found [here](data/snomed)

# Part 1: Preprocessing SNOMED CT for MedCAT

Once you have downloaded a SNOMED release of interest. Store the zipped folder containing your respective SNOMED release in the current colab working directory.

The folder name should look like: `SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z.zip
`


### Import required packages

In [ ]:
from medcat.utils.preprocess_snomed import Snomed

### Load the data
Please see the section: [Access to SNOMED CT release files](#access_to_snomed_ct) for how to retrieve the zipped SNOMED CT release.

In [ ]:
# Assign a path to the zipped SNOMED CT release download. (skip this step if the folder is not zipped)
snomed_path = "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z.zip"  # Enter your zipped Snomed folder here


In [ ]:
!unzip snomed_path

### Preprocess the release for MedCAT

In [ ]:
# Initialise
snomed_filename = "SnomedCT_InternationalRF2_PRODUCTION_20210131T120000Z"  # The unzippedSNOMED CT folder
snomed = Snomed(snomed_filename)

In [ ]:
### Skip this step if your version of snomed is not the UK extension released >2021.
### Note: this step will only work with MedCAT v1.2.7+

# snomed.uk_ext = True

#### Create a SNOMED DataFrame

We first preprocess SNOMED to fit the following format:


|cui|name|ontologies|name_status|description_type_ids|type_ids|
|--|--|--|:--:|:--:|--|
|101009|Quilonia ethiopica (organism)|SNOMED|P|organism|81102976|
.
.
.

`cui` - The concept unique identifier, this is simply the `SCTID`.

`name` - This include the name of the concept. The status of the name is given in `name_status`

`ontologies` - Always SNOMED. Alternatively you can change it to your specific edition.

`name_status` - The Fully specified name or FSN is denoted with a `P` - Primary Name. Each concept must be assigned only one Primary Name. These should be unique across all SCTID/cui to avoid confusion. A synonym or other description type is represented as a `A` - Alternative Name. This can be enriched with all possible names and abbreviations for a concept of interest.

`description_type_ids` - These are processed to be the Semantic Tags of the concept.

`type_ids` - This is simply a 10 digit Hash of the Semantic Tags




In [ ]:
# Create SNOMED DataFrame 
df = snomed.to_concept_df()

In [ ]:
df.head()

In [ ]:
# inspect
df[df['cui'] == '101009']

In [ ]:
# Optional - Create a SCTID to FSN dictionary
primary_names_only = df[df["name_status"] == 'P']
sctid2name = dict(zip(primary_names_only['cui'], primary_names_only['name']))
del primary_names_only

In [ ]:
sctid2name['101009']

#### SNOMED Relationships

In [ ]:
all_snomed_relationships = snomed.list_all_relationships()

In [ ]:
# List of the SCTID of all snomed relationships
all_snomed_relationships

In [ ]:
# Using the SCTID to name to inspect what the FSN (fully specified names) are:
for sctid in all_snomed_relationships:
    print(sctid2name[sctid])

In [ ]:
# save a specific relationship to json
# In the example we save the "IS a (attribute)" hierarchical relationship.
snomed.relationship2json("116680003", "ISA_relationship.json")

#### Mappings to inbuilt external terminologies 

Create a dictionary map to add to the medcat concept database additional information

##### ICD-10
For SNOMED to ICD-10 mapping read more on:
Map Blocks, Map Groups and Map Priorities, for correct official mapping methodology.

In [ ]:
# ICD-10
icd_df = snomed.map_snomed2icd10()

In [ ]:
icd_df.head()

In [ ]:
sctid2icd10 = {k: g["mapTarget"].tolist() for k,g in icd_df[['referencedComponentId',
                                                              'mapTarget']].groupby("referencedComponentId")}

In [ ]:
# To view the SNOMED to ICD-10 Map structure
sctid2icd10

##### OPCS
Office of Population Censuses and Surveys


__Note:__ only the SNOMED UK extension edition contains this information
Skip if your version is not a UK extension

In [ ]:
opcs_df = snomed.map_snomed2opcs4()

In [ ]:
opcs_df.head()

### Save for MedCAT

In [ ]:
# Save to CSV for medcat CDB creation
df.to_csv("preprocessed_snomed.csv", index=False)

--------

# Part 2: Create a MedCAT CDB using SNOMED CT release files


These steps are also in the [create_cdb.py](medcat/1_create_model/create_cdb/create_cdb.py)

In [ ]:
# Import required packages
from medcat.cdb import CDB
from medcat.config import Config
from medcat.cdb_maker import CDBMaker

#### Create concept database (cdb)

In [ ]:
# First initialise the default configuration
config = Config()
config.general['spacy_model'] = 'en_core_web_md'
maker = CDBMaker(config)

In [ ]:
# Create an array containing CSV files that will be used to build our CDB
csv_path = ['preprocessed_snomed.csv']

# Create your CDB
cdb = maker.prepare_csvs(csv_path, full_build=True)

### Inspect your cdb

In [ ]:
print(cdb.name2cuis['epilepsy'])

In [ ]:
print(cdb.cui2preferred_name['84757009'])

In [ ]:
print(cdb.cui2names['84757009'])

#### Enrich with extra information and mapping

Mapping was created in [Mappings to inbuilt external terminologies](https://colab.research.google.com/drive/1yesqjMQwQH20Kl9w7siRGVaSWU0uI84W#scrollTo=Mappings_to_inbuilt_external_terminologies).
Here we use [ICD-10](https://colab.research.google.com/drive/1yesqjMQwQH20Kl9w7siRGVaSWU0uI84W#scrollTo=ICD_10) as an example.

In [ ]:
cdb.addl_info['cui2icd10'] = sctid2icd10

### Save your new SNOMED cdb

__tip:__ good practise to include the snomed release edition file name

In [ ]:
cdb.save("SNOMED_cdb.dat")